In [33]:
import numpy as np
import pandas as pd
import pickle as pkl
import requests as req
g_dict=pkl.load(open("g_dict.pkl","rb"))
sp_dict=pkl.load(open("sp_dict.pkl","rb"))
no2_dict=pkl.load(open("dates_dict.pkl","rb"))

In [102]:
import pprint
pprint.pprint(len([i for i in g_dict.keys()]))

186


In [135]:
c2pop["Afghanistan"]

31575018

In [35]:
a=req.get("https://simple.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population").content
open("pops.html","wb+").write(a)
d=pd.read_html("pops.html")

In [37]:
pops_=d[0][["Country(or dependent territory)","Population"]]

reduction_in_activity_percentage={} # reduction in activity since g_dict is more than 0.012% of the population
multiply g_dict by (sp_dict+reduction_in_activity_percentage)
then scale between 0-10


In [92]:
def ignore_timestamp_zone(stamp):
    return pd.Timestamp("{0}-{1}-{2}".format(stamp.year,stamp.month,stamp.day))
def add_utc_stamp(stamp):
    return pd.Timestamp("{0}-{1}-{2}".format(stamp.year,stamp.month,stamp.day),tz="UTC")

In [165]:
it_is_srys={}
for country in g_dict.keys():
    try:
        pop_thresh=500#c2pop[country]*0.00012
        
    except KeyError:
        print("skipping",country)
        continue
    for date in g_dict[country].keys():
        if g_dict[country][date]>pop_thresh:
            if country not in it_is_srys.keys():
                    it_is_srys.update({country:date})
                    
for country in g_dict.keys():
    if country not in it_is_srys.keys():
            it_is_srys.update({country:np.nan})
            

In [93]:
new_no2_dict={} #invert the dict so it's name->dates->no2_mean instead of dates->names->no2_mean
for date in no2_dict.keys():
    for c_name in no2_dict[date].keys():
        utc_date=add_utc_stamp(date)
        if c_name in new_no2_dict.keys():
            new_no2_dict[c_name].update({utc_date:no2_dict[date][c_name]})
        else:
            new_no2_dict.update({c_name:{utc_date:no2_dict[date][c_name]}})

In [207]:
b4_turning_point={}
after_turning_point={}

for c_name in new_no2_dict.keys():
    date2meanNo2=new_no2_dict[c_name]
    means_b4=np.zeros(shape=(0,))
    means_after=np.zeros(shape=(0,))
    nd=False
    for date in date2meanNo2.keys():
        try:
            if np.isnan(it_is_srys[c_name]):
                nd=True
                continue
        except:
            pass
        if date>=it_is_srys[c_name]: #if it's after it being srys
            means_b4=np.hstack([means_b4,[date2meanNo2[date]]])
        else:
            means_after=np.hstack([means_after,[date2meanNo2[date]]])
    
    means_b4=means_b4[np.logical_not(np.isnan(means_b4))]
    means_after=means_after[np.logical_not(np.isnan(means_after))]
    
    if nd==False:
        b4_turning_point.update({c_name:np.mean(means_b4)})
        after_turning_point.update({c_name:np.mean(means_after)})
    else:
        b4_turning_point.update({c_name:0})
        after_turning_point.update({c_name:0})
    

In [228]:
act_diff_ratio={} # the lower the score the better
for c_name in b4_turning_point.keys():
    if b4_turning_point[c_name]==0:
        act_diff_ratio.update({c_name:0})
        continue
    act_diff_ratio.update({c_name: \
            (b4_turning_point[c_name]-after_turning_point[c_name])
            })

In [268]:
CASPER_INDEX={}
w_act_diff_ratio=.6
w_search_pen=.4
w_growth=.8

def calc_casper(act_diff_ratio,growth,search_pen):
    return (w_growth*sum(growth))*((w_search_pen*search_pen)+(w_act_diff_ratio*act_diff_ratio))
    
for c_name in act_diff_ratio.keys():
    
    growth_over_time=[i[1] for i in g_dict[c_name].items()]
    try:
        search_pen=sp_dict[c_name]
    except KeyError:
        pass
    casper_score=calc_casper(act_diff_ratio[c_name],growth_over_time,search_pen)
    CASPER_INDEX.update({c_name:casper_score})

In [269]:
CASPER_INDEX

{'Afghanistan': 839100.2227548759,
 'Albania': 0.0,
 'Algeria': 0.0,
 'Andorra': 0.0,
 'Angola': 0.0,
 'Antigua and Barbuda': 0.0,
 'Argentina': -3332.245821658258,
 'Armenia': 0.0,
 'Australia': 3482699.837250848,
 'Austria': -11741.637003741758,
 'Azerbaijan': 0.0,
 'Bahamas': 0.0,
 'Bahrain': 0.0,
 'Bangladesh': 2556761.362562502,
 'Barbados': 0.0,
 'Belarus': -17938.73083996691,
 'Belgium': -57277.77360746341,
 'Belize': 0.0,
 'Benin': 0.0,
 'Bhutan': 0.0,
 'Bolivia': -1070.6059697676103,
 'Bosnia and Herzegovina': 0.0,
 'Botswana': 0.0,
 'Brazil': 27989432.140533265,
 'Brunei Darussalam': 9024.0,
 'Bulgaria': 0.0,
 'Burkina Faso': 0.0,
 'Burundi': 0.0,
 'Cambodia': 0.0,
 'Cameroon': 346788.99698854,
 'Canada': 36591255.9551632,
 'Cape Verde': 0.0,
 'Central African Republic': 0.0,
 'Chad': 0.0,
 'Chile': -13190.673601183977,
 'China': 493445.2452484391,
 'Colombia': 1818.1276384680336,
 'Comoros': 0.0,
 'Congo (Brazzaville)': 0.0,
 'Congo (Kinshasa)': 0.0,
 'Costa Rica': 0.0,
 "Cô

In [194]:
act_diff_per["Egypt"]

3.247660925393389

In [195]:
act_diff_per["United States of America"]

4.528118307313536

In [185]:
b4_turning_point["United States of America"]

4.192652273355415

In [186]:
after_turning_point["United States of America"]

4.633192711049197

In [133]:
c2pop={}
import Levenshtein as lev

g_name=[i for i in g_dict.keys()]
c=np.asarray(pops_)
for i in c:
    ds=[]
    for g_n in g_name:
        d = lev.distance(g_n.lower(),i[0].lower()),
        ds.append(d)
    closest_rel_idx=ds.index(min(ds))
    print(i[0],g_name[closest_rel_idx],min(ds),i[1])
    if g_name[closest_rel_idx] not in c2pop.keys():
        c2pop.update({g_name[closest_rel_idx]:i[1]})

China[Note 1] China (8,) 1402870000
India[Note 2] India (8,) 1363010000
United States[Note 3] United States of America (10,) 331362000
Indonesia Indonesia (0,) 268074600
Brazil Brazil (0,) 211563000
Pakistan Pakistan (0,) 208857000
Nigeria Nigeria (0,) 200962417
Bangladesh Bangladesh (0,) 168698000
Russia[Note 4] Guinea (10,) 146793744
Mexico Mexico (0,) 126577691
Japan Japan (0,) 126200000
Philippines Philippines (0,) 109483000
Egypt Egypt (0,) 100605100
Ethiopia Ethiopia (0,) 98665000
Vietnam Viet Nam (1,) 95354000
Democratic Republic of the Congo Taiwan, Republic of China (16,) 86727573
Germany Germany (0,) 82979100
Iran Iraq (1,) 83489300
Turkey Turkey (0,) 82003882
France[Note 5] France (8,) 66992000
Thailand Thailand (0,) 66511080
United Kingdom[Note 6] United Kingdom (8,) 66040229
Italy Italy (0,) 60375749
South Africa South Africa (0,) 57725600
Tanzania[Note 7] Albania (11,) 55890747
Myanmar Myanmar (0,) 54339766
Kenya Kenya (0,) 52214791
South Korea South Africa (4,) 51811167


In [270]:
CASPER_INDEX_scaled={}
casper_array=np.array([[i[0],float(i[1])] for i in CASPER_INDEX.items()])
def rescale_linear(array, new_min, new_max):
    """Rescale an arrary linearly."""
    minimum, maximum = np.min(array), np.max(array)
    m = (new_max - new_min) / (maximum - minimum)
    b = new_min - m * minimum
    return m * array + b

x=np.array([float(i) for i in casper_array[:,1]])
c_idx_keys=[i for i in casper_array[:,0]] #[i for i in CASPER_INDEX.keys()]
for i in range(len(c_idx_keys)):
    CASPER_INDEX_scaled.update({c_idx_keys[i]:rescale_linear(x,0,1000)[i]})


In [271]:
CASPER_INDEX_scaled

{'Afghanistan': 779.543956304831,
 'Albania': 774.3736076132578,
 'Algeria': 774.3736076132578,
 'Andorra': 774.3736076132578,
 'Angola': 774.3736076132578,
 'Antigua and Barbuda': 774.3736076132578,
 'Argentina': 774.3530750566492,
 'Armenia': 774.3736076132578,
 'Australia': 795.8332283574325,
 'Austria': 774.3012582610911,
 'Azerbaijan': 774.3736076132578,
 'Bahamas': 774.3736076132578,
 'Bahrain': 774.3736076132578,
 'Bangladesh': 790.127801697713,
 'Barbados': 774.3736076132578,
 'Belarus': 774.2630731490643,
 'Belgium': 774.0206747289651,
 'Belize': 774.3736076132578,
 'Benin': 774.3736076132578,
 'Bhutan': 774.3736076132578,
 'Bolivia': 774.3670107777132,
 'Bosnia and Herzegovina': 774.3736076132578,
 'Botswana': 774.3736076132578,
 'Brazil': 946.8382547689507,
 'Brunei Darussalam': 774.4292114914687,
 'Bulgaria': 774.3736076132578,
 'Burkina Faso': 774.3736076132578,
 'Burundi': 774.3736076132578,
 'Cambodia': 774.3736076132578,
 'Cameroon': 776.5104441772442,
 'Canada': 999.84